In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
%pylab inline
#import shapefile

Populating the interactive namespace from numpy and matplotlib


In [3]:
car = pd.read_csv('https://serv.cusp.nyu.edu/~cq299/ADS2016/Data/Car.csv')
car.head()

,Price,Mileage,Make,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.103129,8221,Buick,6,3.1,4,1,1,1
1,17542.036083,9135,Buick,6,3.1,4,1,1,0
2,16218.847862,13196,Buick,6,3.1,4,1,1,0
3,16336.913140,16342,Buick,6,3.1,4,1,0,0
4,16339.170324,19832,Buick,6,3.1,4,1,0,1


In [9]:
np.random.seed(1)
shuf = np.random.permutation(len(car))
train = car.take(shuf)[:int(0.7*(len(car)))]
test = car.take(shuf)[int(0.7*(len(car))):]

In [117]:
# An anonymous function that returns your regression model for future usage
Regress = lambda feature: smf.ols(formula = 'Price ~ %s'%feature, data = train).fit()

# For displaying R^2 and linear model summary
def modelEval(lm, key = 'Price', plot = False, summary = False):
    lmy = lm.predict(test)
    y_err = lmy - test[key]
    y_norm = test[key]-mean(test[key])
    R2 = 1-y_err.dot(y_err)/y_norm.dot(y_norm)
    if summary:
        print('Out of sample R^2 is %f'%R2)
        print(lm.summary())
    if plot:
        plt.plot(lmy, lmy, 'b-')
        plt.plot(lmy, test[key],'or')
    return R2

def modelEval2(flag):
    return modelEval(Regress('+'.join([feat for feat in selection if flag[list(selection).index(feat)]])))

## 1. Multi-variate regression

In [131]:
print(Regress('+'.join(selection)).summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     349.2
Date:                Thu, 03 Nov 2016   Prob (F-statistic):          7.93e-248
Time:                        10:36:07   Log-Likelihood:                -5366.9
No. Observations:                 562   AIC:                         1.076e+04
Df Residuals:                     549   BIC:                         1.082e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Intercept          1.637e+04   1512.23

## 2. Feature selection: step-forwad--keep adding features until out-of-sample $R^2$ stops increasing

In [105]:
selection = car.columns[1:]
features = []
for i in selection:
    print('Out of sample R^2 after adding %s is %f'%(i,modelEval(Regress(i))))

Out of sample R^2 after adding Mileage is 0.019001
Out of sample R^2 after adding Make is 0.679142
Out of sample R^2 after adding Cylinder is 0.345937
Out of sample R^2 after adding Liter is 0.315642
Out of sample R^2 after adding Doors is -0.007377
Out of sample R^2 after adding Cruise is 0.171303
Out of sample R^2 after adding Sound is 0.017541
Out of sample R^2 after adding Leather is -0.001127


In [106]:
features.append('Make')
for i in selection:
    if i in features:
        continue
    print('Out of sample R^2 after adding %s is %f'%(i,modelEval(Regress('+'.join(features)+'+'+i))))

Out of sample R^2 after adding Mileage is 0.705224
Out of sample R^2 after adding Cylinder is 0.816120
Out of sample R^2 after adding Liter is 0.826723
Out of sample R^2 after adding Doors is 0.728122
Out of sample R^2 after adding Cruise is 0.696199
Out of sample R^2 after adding Sound is 0.672732
Out of sample R^2 after adding Leather is 0.679712


In [107]:
features.append('Liter')
for i in selection:
    if i in features:
        continue
    print('Out of sample R^2 after adding %s is %f'%(i,modelEval(Regress('+'.join(features)+'+'+i))))

Out of sample R^2 after adding Mileage is 0.854135
Out of sample R^2 after adding Cylinder is 0.824535
Out of sample R^2 after adding Doors is 0.837494
Out of sample R^2 after adding Cruise is 0.826961
Out of sample R^2 after adding Sound is 0.826228
Out of sample R^2 after adding Leather is 0.826667


In [108]:
features.append('Mileage')
for i in selection:
    if i in features:
        continue
    print('Out of sample R^2 after adding %s is %f'%(i,modelEval(Regress('+'.join(features)+'+'+i))))

Out of sample R^2 after adding Cylinder is 0.852176
Out of sample R^2 after adding Doors is 0.860400
Out of sample R^2 after adding Cruise is 0.854320
Out of sample R^2 after adding Sound is 0.853866
Out of sample R^2 after adding Leather is 0.854106


In [109]:
features.append('Doors')
for i in selection:
    if i in features:
        continue
    print('Out of sample R^2 with %s is %f'%(i,modelEval(Regress('+'.join(features)+'+'+i))))

Out of sample R^2 with Cylinder is 0.859931
Out of sample R^2 with Cruise is 0.861185
Out of sample R^2 with Sound is 0.859705
Out of sample R^2 with Leather is 0.860431


In [110]:
features.append('Cruise')
for i in selection:
    if i in features:
        continue
    print('Out of sample R^2 with %s is %f'%(i,modelEval(Regress('+'.join(features)+'+'+i))))

Out of sample R^2 with Cylinder is 0.860692
Out of sample R^2 with Sound is 0.860500
Out of sample R^2 with Leather is 0.861185


## Rewrite it in matrix computation format

In [129]:
def FeatureSelection():
    n = len(selection)
    flag = np.zeros(n)    # initially exclude all regressors: 1-include, 0-exclude
    r2max = 0             # for storing and compare with best R^2
    while True:
        flag_mat = np.maximum(np.eye(n),flag)
        # see if R2 increases if we add one more feature
        r2 = np.apply_along_axis(modelEval2,1,flag_mat)  # 1: row-wise operation, then select R2 only
        temp = r2.max()
        if temp > r2max:
            r2max = temp
            flag = flag_mat[r2.argmax()]  # select this feature if it improves our R2
        else:
            break                         # if there's nothing to add, break the loop
    features2 = [feat for feat in selection if flag[list(selection).index(feat)]]
    return features2

In [130]:
FeatureSelection()

['Mileage', 'Make', 'Liter', 'Doors', 'Cruise']